In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [0]:
train_df = pd.read_csv("../input/train_V2.csv")
test_df = pd.read_csv("../input/test_V2.csv")

In [0]:
train_df.head()

In [0]:
train_df.isnull().sum()

In [0]:
matchTyp = ['squad-fpp', 'duo', 'solo-fpp', 'squad', 'duo-fpp', 'solo',
       'normal-squad-fpp', 'crashfpp', 'flaretpp', 'normal-solo-fpp',
       'flarefpp', 'normal-duo-fpp', 'normal-duo', 'normal-squad',
       'crashtpp', 'normal-solo']
mapping = {}
for i, j in enumerate(matchTyp):
    mapping[i] = j
train_df["matchTypeMap"] = train_df["matchType"].apply(lambda x: ''.join(str(i) for i, j in mapping.items() if x == j)).map(int)
test_df["matchTypeMap"] = test_df["matchType"].apply(lambda x: ''.join(str(i) for i, j in mapping.items() if x == j)).map(int)

In [0]:
train_df.drop(["matchType"], axis =1, inplace=True)
test_df.drop(["matchType"], axis =1, inplace=True)

In [0]:
train_df.dropna(inplace = True)
train_df.isnull().any().any()

In [0]:
def rank_by_team(df):
    cols_to_drop = ['Id', 'groupId', 'matchId', 'winPlacePerc', 'matchTypeMap']
    features = [col for col in df.columns if col not in cols_to_drop]
    agg = df.groupby(['matchId', 'groupId'])[features].mean()
    agg = agg.groupby('matchId')[features].rank(pct=True)
    return df.merge(agg, suffixes=['', '_mean_rank'], how='left', on=['matchId', 'groupId'])

In [0]:
train_df = rank_by_team(train_df)
test_df = rank_by_team(test_df)

In [0]:
X = train_df.drop(["Id", "groupId", "matchId", "winPlacePerc"], axis = 1)
y = train_df["winPlacePerc"]
test = test_df.drop(["Id", "groupId", "matchId"], axis = 1)

In [0]:
X.head()

In [0]:
del train_df

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [0]:
model = xgb.XGBRegressor(max_depth=5, gamma=0.1, learning_rate= 0.1, n_jobs=10)
model.fit(X_train,y_train)

In [0]:
from sklearn.metrics import mean_absolute_error
y_pred=model.predict(X_test)
print('mean absolute error is:',mean_absolute_error(y_test,y_pred))

In [0]:
ax = xgb.plot_importance(model)
fig = ax.figure
fig.set_size_inches(15, 10)

In [0]:
pred = model.predict(test)

In [0]:
test_id = test_df["Id"]
submit_xg = pd.DataFrame({'Id': test_id, "winPlacePerc": pred} , columns=['Id', 'winPlacePerc'])
print(submit_xg.head())
submit_xg.to_csv("submission.csv", index = False)